In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('inreality').getOrCreate()

server_name = "jdbc:sqlserver://inreality.database.windows.net:1433"
database_name = "data"
url = server_name + ";" + "databaseName=" + database_name + ";"

table_name = "dbo.review"
username = ""
password = "" # Please specify password here

jdbcDF = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password)\
        .load()


22/10/07 19:44:44 WARN Utils: Your hostname, TamdeMacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.100 instead (on interface en0)
22/10/07 19:44:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/07 19:44:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/07 19:44:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Py4JJavaError: An error occurred while calling o30.load.
: java.lang.ClassNotFoundException: 
Failed to find data source: com.microsoft.sqlserver.jdbc.spark. Please find packages at
https://spark.apache.org/third-party-projects.html
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedToFindDataSourceError(QueryExecutionErrors.scala:574)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:675)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:725)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:207)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:171)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java

In [ ]:
df = jdbcDF.toPandas()
experimentDF = df[['prodcut_name','reviewerName','overall']]
display(experimentDF)

In [ ]:
import pandas as pd

pivot_product_df = pd.pivot_table(experimentDF, index=['prodcut_name'],columns=['reviewerName'],values=['overall'],fill_value=0)
pivot_product_df_numpy = pd.pivot_table(experimentDF, index=['prodcut_name'],columns=['reviewerName'],values=['overall'],fill_value=0).to_numpy()
pivot_user_df = pd.pivot_table(experimentDF, index=['reviewerName'],columns=['prodcut_name'],values=['overall'],fill_value=0)
pivot_user_df_numpy = pd.pivot_table(experimentDF, index=['reviewerName'],columns=['prodcut_name'],values=['overall'],fill_value=0).to_numpy()

In [ ]:
import numpy as np
print(pivot_product_df_numpy.shape)
print(pivot_user_df_numpy.shape)

In [ ]:
train_product = pivot_product_df_numpy[:,:int(2734 * 0.8)]
test_product = pivot_product_df_numpy[:,int(2734 * 0.2):]
train_user = pivot_user_df_numpy[:,:int(2112 * 0.8)]
test_user = pivot_user_df_numpy[:,int(2112 * 0.2):]

In [ ]:
from sklearn.decomposition import TruncatedSVD
product_svd = TruncatedSVD(n_components=10)
product_features = product_svd.fit_transform(train_product)
user_svd = TruncatedSVD(n_components=10)
user_features = user_svd.fit_transform(train_user)

In [ ]:
print(product_features.shape) 
print(user_features.shape)

print(product_features)

In [ ]:
corr_between_diff_product = np.corrcoef(product_features)
corr_fifa = corr_between_diff_product[1089]
recommand = list(pivot_product_df.index[corr_fifa>0.7])
recommand.remove('Lego Star Wars: The Complete Saga - Xbox 360')
print(recommand[:10])

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline
tsne = TSNE(init='pca')
plt.figure(figsize=(100, 100))
coordination = tsne.fit_transform(product_features[:,:])
labels = [productname for productname in list(pivot_product_df.index)]
for i, label in enumerate(labels):
    x, y = coordination[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(10, 4),
                 textcoords="offset points",
                 ha="right",
                 va="bottom")
plt.show()